In [1]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [3]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet...
- [1 files][  7.1 GiB/  7.1 GiB]                                                
Operation completed over 1 objects/7.1 GiB.                                      


In [11]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'
base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01.parquet'

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)

In [12]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [13]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105, 202106]

data = data[data['foto_mes'].isin(meses_train)]

In [14]:
data.shape

(3773357, 679)

In [15]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201906,0.995294,0.004706
1,201907,0.994729,0.005271
2,201908,0.995839,0.004161
3,201909,0.995712,0.004288
4,201910,0.995435,0.004565
5,201911,0.994700,0.005300
6,201912,0.995749,0.004251
7,202001,0.996513,0.003487
8,202002,0.998742,0.001258
9,202003,0.997469,0.002531


In [16]:
df_subsampled = []

for mes, group in data.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    print(f'Original proportion:{minority_proportion}')
    
    # voy a incrementar esa proporcion por 10
    new_proportion = minority_proportion * 10

    # Apply RandomUnderSampler with the calculated strategy
    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rebuild the resampled DataFrame for the group
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Add back the 'foto_mes' column

    # Add the resampled DataFrame to the list
    df_subsampled.append(group_resampled)

# Concatenate all resampled groups into a single DataFrame
data = pd.concat(df_subsampled, ignore_index=True)

Original proportion:0.004706392333534593
Original proportion:0.005270646553043053
Original proportion:0.004160887656033287
Original proportion:0.004288458388552198
Original proportion:0.00456534144949591
Original proportion:0.005300468027721088
Original proportion:0.00425135609728354
Original proportion:0.003486934415070225
Original proportion:0.0012575709168031867
Original proportion:0.0025308658507190874
Original proportion:0.003556368100779332
Original proportion:0.0041583752586588744
Original proportion:0.004058351814876721
Original proportion:0.003479623019439665
Original proportion:0.003005259203606311
Original proportion:0.003561258058609215
Original proportion:0.003055136448153458
Original proportion:0.004018862525040127
Original proportion:0.003925064695466983
Original proportion:0.004844901435572068
Original proportion:0.006252843598981838
Original proportion:0.0059932186822250055
Original proportion:0.007246023523676031
Original proportion:0.0055338561440381965
Original prop

In [17]:
data.shape

(393861, 679)

In [18]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201906,0.955050,0.044950
1,201907,0.949931,0.050069
2,201908,0.960052,0.039948
3,201909,0.958878,0.041122
4,201910,0.956339,0.043661
5,201911,0.949661,0.050339
6,201912,0.959220,0.040780
7,202001,0.966304,0.033696
8,202002,0.987580,0.012420
9,202003,0.975315,0.024685


In [19]:
output_file = 'competencia_02_fe_v01_undersampled_10_24M_altbinaria.parquet'

data.to_parquet('/home/eanegrin/datasets/' + output_file, index=False)

In [20]:
!gsutil cp /home/eanegrin/datasets/competencia_02_fe_v01_undersampled_10_24M_altbinaria.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_02_fe_v01_undersampled_10_24M_altbinaria.parquet...
- [1 files][607.0 MiB/607.0 MiB]                                                
Operation completed over 1 objects/607.0 MiB.                                    
